In [12]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import os
import time
from tensorflow.keras import layers

# Create a directory to store the generated images
os.makedirs('generated_images', exist_ok=True)

# Load the dataset
dataset, info = tfds.load('oxford_flowers102', split='train', with_info=True, as_supervised=True)

# Function to preprocess the images
def preprocess_image(image, label):
    image = tf.image.resize(image, [64, 64])  # Resize images to 64x64
    image = tf.cast(image, tf.float32)  # Convert image to float32
    image = (image - 127.5) / 127.5  # Normalize the images to [-1, 1]
    return image

# Apply the preprocessing function to the dataset
dataset = dataset.map(lambda image, label: preprocess_image(image, label))
dataset = dataset.batch(10)  # Batch all 10 images together

# Prefetch the dataset for performance
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [13]:

# Generator Model
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((8, 8, 256)))
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

# Discriminator Model
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[64, 64, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

# Define the loss function for the discriminator
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)



In [14]:

# Define the optimizers for both models
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Initialize the generator and discriminator
generator = make_generator_model()
discriminator = make_discriminator_model()

# Define the number of epochs and seed for generating images
EPOCHS = 1000
noise_dim = 100
num_examples_to_generate = 10  # Generate 10 images

# Seed to visualize the progress
seed = tf.random.normal([num_examples_to_generate, noise_dim])

# Define the training step
@tf.function
def train_step(images):
    noise = tf.random.normal([10, noise_dim])  # Match the batch size of 10

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Function to save the generated images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(2, 5, i+1)  # Adjust the subplot layout to fit 10 images
        plt.imshow((predictions[i] * 127.5 + 127.5).numpy().astype(int))
        plt.axis('off')

    plt.savefig(f'generated_images/final_image.png')
    plt.close(fig)

# Define the training function with only final image saving
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        print(f'Time for epoch {epoch + 1} is {time.time() - start} sec')

    # Generate after the final epoch
    generate_and_save_images(generator, epochs, seed)

# Train the GAN
train(dataset, EPOCHS)

Time for epoch 1 is 5.483246803283691 sec
Time for epoch 2 is 1.4772188663482666 sec
Time for epoch 3 is 1.4794285297393799 sec
Time for epoch 4 is 1.4817705154418945 sec
Time for epoch 5 is 1.4820551872253418 sec
Time for epoch 6 is 1.4781467914581299 sec
Time for epoch 7 is 2.2644577026367188 sec
Time for epoch 8 is 2.051278591156006 sec
Time for epoch 9 is 1.4946467876434326 sec
Time for epoch 10 is 1.4808268547058105 sec
Time for epoch 11 is 1.4755785465240479 sec
Time for epoch 12 is 1.4942288398742676 sec
Time for epoch 13 is 1.4892008304595947 sec
Time for epoch 14 is 1.4724853038787842 sec
Time for epoch 15 is 2.5137197971343994 sec
Time for epoch 16 is 1.7299306392669678 sec
Time for epoch 17 is 1.5246832370758057 sec
Time for epoch 18 is 1.4704785346984863 sec
Time for epoch 19 is 1.4915194511413574 sec
Time for epoch 20 is 1.4846301078796387 sec
Time for epoch 21 is 1.4806532859802246 sec
Time for epoch 22 is 1.5616281032562256 sec
Time for epoch 23 is 2.79646635055542 sec
T